In [ ]:
!pip3 install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=2b6d9f5775cf6905e6987786f9ecd414a8fe3898894ee3b65481c753160a1c24
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import wget

## read dataset and pandas

In [ ]:
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt')
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')
wget.download('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')

'ratings_train.txt'

In [ ]:
import pandas as pd
train_df = pd.read_csv('ratings_train.txt', sep='\t')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [ ]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [ ]:
train_df['document'].isna().value_counts()

False    149995
True          5
Name: document, dtype: int64

In [ ]:
import re
train_df = train_df.fillna(' ')
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r'\d+',' ',x))

## read test dataset

In [ ]:
test_df = pd.read_csv('ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x : re.sub(r'\d+',' ',x))
test_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  50000 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


## sklearen

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer


## import KoNLPy

In [ ]:
!pip3 install konlpy

     |████████████████████████████████| 19.4MB 163kB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 3.6MB 52.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
okt.morphs('아 더빙.. 진짜 짜증나네요. 목소리')

['아', '더빙', '..', '진짜', '짜증나네요', '.', '목소리']

In [ ]:
tfidfVectorizer = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)

In [ ]:
import datetime
running_time = datetime.datetime.now()
print('start time : ' + str(running_time))

tfidVec_matrix_train = tfidfVectorizer.fit_transform(train_df['document'])

print('end time : ' + str(datetime.datetime.now()))
print('running time : ' + str(datetime.datetime.now() - running_time))

start time : 2020-05-28 01:33:24.639029
end time : 2020-05-28 01:40:27.061273
running time : 0:07:02.422529


In [ ]:
tfidVec_matrix_train

<150000x129276 sparse matrix of type '<class 'numpy.float64'>'
	with 3069141 stored elements in Compressed Sparse Row format>

## Training LogisticRegresstion with GridSearchCV

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
logisticRegression = LogisticRegression(random_state=0)

## GridSearchCV

In [ ]:
running_time = datetime.datetime.now()
print('start time : ' + str(running_time))

params = {'C':[1,3.5,4.5,5.5,10]}
gridSearchCV = GridSearchCV(logisticRegression, param_grid=params, cv=3, scoring='accuracy', verbose=1)
gridSearchCV.fit(tfidVec_matrix_train, train_df['label'])
print(gridSearchCV.best_params_, round(gridSearchCV.best_score_,4))

print('end time : ' + str(datetime.datetime.now()))
print('running time : ' + str(datetime.datetime.now() - running_time))

In [ ]:
running_time = datetime.datetime.now()
print('start time : ' + str(running_time))

tfidVec_matrix_test = tfidfVectorizer.transform(test_df['document'])

print('end time : ' + str(datetime.datetime.now()))
print('running time : ' + str(datetime.datetime.now() - running_time))

In [ ]:
type(test_df['document'])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
test_predict = gridSearchCV.predict(tfidVec_matrix_test)
test_predict

In [ ]:
print('LogisticRegression accuracy_score', accuracy_score(test_df['label'], test_predict))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_df['label'], test_predict))

## Dump and Load a dictionary into a pickle file.

In [ ]:
import pickle

pickle.dump( gridSearchCV, open( "NaverMovieRatingGridSearchCVByLogisticRegression.pkl", "wb" ) )

In [ ]:
pkl_load = pickle.load( open( "NaverMovieRatingGridSearchCVByLogisticRegression.pkl", "rb" ) )

In [ ]:
pkl_predict = pkl_load.predict(tfidVec_matrix_test)

In [ ]:
print('LogisticRegression accuracy_score', accuracy_score(test_df['label'], pkl_predict))

In [ ]:
tfidVec_matrix_predict = tfidfVectorizer.transform(['정말 엉망으로 만들어 졌다.'])
print(tfidVec_matrix_predict)

In [ ]:
pkl_predict = pkl_load.predict(tfidVec_matrix_predict)
pkl_predict

In [ ]:
tfidVec_matrix_predict = tfidfVectorizer.transform(['방금 보고 옴. mvp: 잉그리스 여왕(혼자서 4인분 능가하는 캐리력)트롤러: 오로라(웃으면서 맘 놓게 하고 뒤통수 여러번 침, 도라이)무임탑승자: 존 왕(숙면하고 나니 염원하던 화합 이뤄짐 대박 버스)말레피센트 개고생 시키고 기어이 마지막에 한번 더 구해줬더니 오로라가 쳐웃으면서 well, well 하는 거 보고 도라이 아냐? 라고 소리내서 말함'])
print(tfidVec_matrix_predict)

In [ ]:
pkl_predict = pkl_load.predict(tfidVec_matrix_predict)
pkl_predict

##LogisticRegression

In [ ]:
logisticRegression.fit(tfidVec_matrix_train, train_df['label'])
test_predict = logisticRegression.predict(tfidVec_matrix_test)
accuracy_score(test_df['label'], test_predict)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_df['label'], test_predict))

In [ ]:
lr_predict = logisticRegression.predict(tfidVec_matrix_predict)
lr_predict

#Test with LogisticRegression

In [ ]:
import pandas as pd
train_df = pd.read_csv('ratings_train.txt', sep='\t')
import re
train_df = train_df.dropna()
# train_df = train_df.fillna(' ')
# train_df['document'] = train_df['document'].apply(lambda x : re.sub(r'\d+',' ',x))
train_df.info()
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
okt = Okt()
okt.morphs('아 더빙.. 진짜 짜증나네요. 목소리')
tfidfVectorizer = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidVec_matrix_train = tfidfVectorizer.fit_transform(train_df['document'])
from sklearn.linear_model import LogisticRegression
logisticRegression = LogisticRegression(random_state=0)
logisticRegression.fit(tfidVec_matrix_train, train_df['label'])
tfidVec_matrix_predict = tfidfVectorizer.transform(['정말 엉망으로 만들어 졌다.'])
print(tfidVec_matrix_predict)
logisticRegression.predict(tfidVec_matrix_predict)

#Test with GaussianNB

In [ ]:
import pandas as pd
train_df = pd.read_csv('ratings_train.txt', sep='\t')
import re
train_df = train_df.dropna()
train_df.info()
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
okt = Okt()
okt.morphs('아 더빙.. 진짜 짜증나네요. 목소리')
tfidfVectorizer = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidVec_matrix_train = tfidfVectorizer.fit_transform(train_df['document'])
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(tfidVec_matrix_train, train_df['label'])
tfidVec_matrix_predict = tfidfVectorizer.transform(['정말 엉망으로 만들어 졌다.'])
gnb.predict(tfidVec_matrix_predict)

#Evaluation

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_df['document'],train_df['label'], test_size=0.5)

tfidfVectorizer = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
x_matrix_train = tfidfVectorizer.fit_transform(x_train)
from sklearn.linear_model import LogisticRegression
logisticRegression = LogisticRegression(random_state=0)
logisticRegression.fit(x_matrix_train, y_train)

x_matrix = tfidfVectorizer.transform(x_test)
y_pred = logisticRegression.predict(x_matrix)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

           0       0.84      0.86      0.85     37646
           1       0.86      0.84      0.85     37354

    accuracy                           0.85     75000
   macro avg       0.85      0.85      0.85     75000
weighted avg       0.85      0.85      0.85     75000



              precision    recall  f1-score   support

           0       0.85      0.86      0.85     22467
           1       0.86      0.84      0.85     22533

    accuracy                           0.85     45000
   macro avg       0.85      0.85      0.85     45000
weighted avg       0.85      0.85      0.85     45000



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[32341,  5305],
       [ 5997, 31357]])